## Persistence and Streaming

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [9]:
import operator
from typing import TypedDict, Annotated, Sequence

from langgraph.graph import StateGraph, END
from langgraph.checkpoint import BaseCheckpointSaver

from langchain_core.messages import (
    AnyMessage,
    BaseMessage,
    SystemMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.tools import BaseTool
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
tool = TavilySearchResults(max_results=2) 

In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [6]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [10]:
class Agent:
    def __init__(
        self,
        model: BaseChatModel,
        tools: Sequence[BaseTool],
        checkpointer: BaseCheckpointSaver,
        system: str = "",
    ) -> None:
        self.system = system

        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_llm)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)

        self.tools = {tool.name: tool for tool in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState) -> bool:
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def call_llm(self, state: AgentState) -> dict[str, list[BaseMessage]]:
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def take_action(self, state: AgentState) -> dict[str, list[ToolMessage]]:
        tool_calls = state["messages"][-1].tool_calls
        results: list[ToolMessage] = []

        for t in tool_calls:
            print(f"Calling: {t}")
            if t["name"] not in self.tools:
                print("\n ....bad tool name....")
                result = "bad tool name, retry"
            else:
                result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!")
        
        return {"messages": results}

In [12]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatGroq(temperature=0, model="llama3-70b-8192")
# model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
abot = Agent(model, [tool], checkpointer=memory, system=prompt)

In [13]:
messages = [HumanMessage(content="What is the weather in Ibadan?")]

In [14]:
thread = {"configurable": {"thread_id": "1"}}

In [15]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n6c6', 'function': {'arguments': '{"query":"current weather in Ibadan"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 1016, 'total_tokens': 1067, 'completion_time': 0.143144354, 'prompt_time': 0.195837961, 'queue_time': None, 'total_time': 0.338982315}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_c1a4bcec29', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6b690b75-f24b-42e3-8470-13997875d294-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Ibadan'}, 'id': 'call_n6c6'}])]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Ibadan'}, 'id': 'call_n6c6'}
Back to the model!
[ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Ibadan\', \'region\': \'Oyo\', \'country\': \'N

In [16]:
messages = [HumanMessage(content="What about in Calabar?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mw4r', 'function': {'arguments': '{"query":"current weather in Calabar"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 1565, 'total_tokens': 1609, 'completion_time': 0.122601691, 'prompt_time': 0.290534298, 'queue_time': None, 'total_time': 0.41313598900000004}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_e897b1804a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8ec6e93e-9810-492a-b3d1-ee2556c7374e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Calabar'}, 'id': 'call_mw4r'}])]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Calabar'}, 'id': 'call_mw4r'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Calabar\'

In [17]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Both Ibadan and Calabar have the same temperature, 29.1°C (84.4°F), so they are equally warm.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 2150, 'total_tokens': 2179, 'completion_time': 0.080462717, 'prompt_time': 0.357691134, 'queue_time': None, 'total_time': 0.43815385100000004}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'stop', 'logprobs': None}, id='run-9587c3fa-5734-41e1-b66b-267ef456e77f-0')]}


Different thread id

In [18]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3vwy', 'function': {'arguments': '{"query":"temperature comparison"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 1012, 'total_tokens': 1049, 'completion_time': 0.103642326, 'prompt_time': 0.16940547, 'queue_time': None, 'total_time': 0.273047796}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3cc6703c-0935-49d2-94b6-c41aeef27f0c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'temperature comparison'}, 'id': 'call_3vwy'}])]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'temperature comparison'}, 'id': 'call_3vwy'}
Back to the model!
{'messages': [ToolMessage(content="[{'url': 'https://weatherandclimate.com/compare-climate', 'content': 'Compare Weather & Climate. Compare city weathe

### Streaming tokens

In [21]:
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [22]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events(
    {"messages": messages}, thread, version="v1"
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_uO9bV3PNXOZiSRVWoNnzogsN'}
Back to the model!
The| current| weather| in| San| Francisco| is| sunny| with| a| temperature| of| |57|.|0|°F| (|13|.|9|°C|).| The| wind| speed| is| |3|.|6| km|/h| coming| from| the| north|.| The| humidity| is| at| |83|%,| and| there| is| no| precipitation|.|